<a href="https://colab.research.google.com/github/derek-shing/ProofOfConcept/blob/master/Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Testing the Speech to Text Cantonese API


1.   Preprocessing the audio file


*   How to record sound
*   turn to correct format
*   display codec spec.
*   Save to Storage



2.   Using AutoML API
*    Read from Storage
*    Call the API
*    Store the result to Firestore

https://cloud.google.com/firestore/docs/quickstart-servers


https://cloud.google.com/speech-to-text/docs/quickstart  

Convert mp3 to wav:

https://www.online-convert.com/result/b85182ca-bb4a-47a1-9d58-86e22f150a87







In [0]:
project_id = 'my-project-1558599394309'

In [0]:
from google.colab import auth
auth.authenticate_user()

In [3]:
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [4]:
!gsutil cp gs://food1430/newSong.wav newSong.wav

Copying gs://food1430/newSong.wav...
- [1 files][ 10.1 MiB/ 10.1 MiB]                                                
Operation completed over 1 objects/10.1 MiB.                                     


In [0]:
!ffmpeg -i cantonese_flac.flac -ac 1 mono.flac

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
!wget https://www.signalogic.com/melp/eng_noise/orig/car.wav

--2020-01-24 08:03:49--  https://www.signalogic.com/melp/eng_noise/orig/car.wav
Resolving www.signalogic.com (www.signalogic.com)... 209.150.126.178
Connecting to www.signalogic.com (www.signalogic.com)|209.150.126.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256044 (250K) [audio/x-wav]
Saving to: ‘car.wav’

car.wav             100%[===================>] 250.04K  --.-KB/s    in 0.06s   

2020-01-24 08:03:49 (4.41 MB/s) - ‘car.wav’ saved [256044/256044]



In [0]:
!gsutil cp /content/car.wav gs://food1430/car.wav

Copying file:///content/car.wav [Content-Type=audio/x-wav]...
-
Operation completed over 1 objects/250.0 KiB.                                    


In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/My Drive/GCP_account.json"

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
pip install google-cloud-speech

     |████████████████████████████████| 92kB 5.8MB/s 


In [0]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
#file_name = os.path.join(
#    os.path.dirname(__file__),
#    'resources',
#    'audio.raw')

file_name='/content/newSong.wav'

# Loads the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    language_code='yue-Hant-HK')

# Detects speech in the audio file
response = client.recognize(config, audio)

for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))

In [0]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums


def sample_long_running_recognize(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100

    # The language of the supplied audio
    language_code = "yue-Hant-HK"

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
    return response

In [9]:
uri="gs://food1430/newSong.wav"
data = sample_long_running_recognize(uri)

Waiting for operation to complete...
Transcript: 美式無限無限寬頻人人有份好歡迎大家收聽今日嘅無限寬頻我已經為大家可以畫3d今日呢我就準備咗一間餐廳同你講下因為我覺得你會鍾意食嘅嘢嚟嘅佢每次都話我鍾意食嘅嘢好容易搵嘅啫因為我可以食嘅咁有啲嘢係你冇咁鍾意食㗎嘛甜品呢樣嘢呢其實就先講星期日本日本日本餐廳啦咁後日本餐都好多種㗎嘛食好多嘢嘅我覺得係比較
Transcript: 人哋啲譬如係一個唔係亞洲人啦可能係其他文化嘅人呢都會接受食呢樣嘢因為唔係嗰啲生嘢食食完返去你會驚我哋飲住先我哋約下中國飲茶食嗰啲鳳爪佢哋又會驚㗎咁樣嘢呢就基本上佢哋唔會驚嘅食緊嚟㗎咁所以係比較大就快啲囉應該就唔會拉麵咁普通啦唔係會唔會係日本嘅定食嗰啲嘢嘅舖頭特別在咩呢佢開鋪頭嗰個人下低嗰個主廚個師傅呢佢並唔係日本人哋嘅咁個故事好得意下其實佢個鼻老人


In [12]:
data

results {
  alternatives {
    transcript: "\347\276\216\345\274\217\347\204\241\351\231\220\347\204\241\351\231\220\345\257\254\351\240\273\344\272\272\344\272\272\346\234\211\344\273\275\345\245\275\346\255\241\350\277\216\345\244\247\345\256\266\346\224\266\350\201\275\344\273\212\346\227\245\345\230\205\347\204\241\351\231\220\345\257\254\351\240\273\346\210\221\345\267\262\347\266\223\347\202\272\345\244\247\345\256\266\345\217\257\344\273\245\347\225\2533d\344\273\212\346\227\245\345\221\242\346\210\221\345\260\261\346\272\226\345\202\231\345\222\227\344\270\200\351\226\223\351\244\220\345\273\263\345\220\214\344\275\240\350\254\233\344\270\213\345\233\240\347\202\272\346\210\221\350\246\272\345\276\227\344\275\240\346\234\203\351\215\276\346\204\217\351\243\237\345\230\205\345\230\242\345\232\237\345\230\205\344\275\242\346\257\217\346\254\241\351\203\275\350\251\261\346\210\221\351\215\276\346\204\217\351\243\237\345\230\205\345\230\242\345\245\275\345\256\271\346\230\223\346\2

In [15]:
data.results[0].alternatives[0].transcript

'美式無限無限寬頻人人有份好歡迎大家收聽今日嘅無限寬頻我已經為大家可以畫3d今日呢我就準備咗一間餐廳同你講下因為我覺得你會鍾意食嘅嘢嚟嘅佢每次都話我鍾意食嘅嘢好容易搵嘅啫因為我可以食嘅咁有啲嘢係你冇咁鍾意食㗎嘛甜品呢樣嘢呢其實就先講星期日本日本日本餐廳啦咁後日本餐都好多種㗎嘛食好多嘢嘅我覺得係比較'

In [0]:
s="\344\272\272\345\223\213\345\225\262\350\255\254\345\246\202\344\277\202\344\270\200\345\200\213\345\224\224\344\277\202\344\272\236\346\264\262\344\272\272\345\225\246\345\217\257\350\203\275\344\277\202\345\205\266\344\273\226\346\226\207\345\214\226\345\230\205\344\272\272\345\221\242\351\203\275\346\234\203\346\216\245\345\217\227\351\243\237\345\221\242\346\250\243\345\230\242\345\233\240\347\202\272\345\224\224\344\277\202\345\227\260\345\225\262\347\224\237\345\230\242\351\243\237\351\243\237\345\256\214\350\277\224\345\216\273\344\275\240\346\234\203\351\251\232\346\210\221\345\223\213\351\243\262\344\275\217\345\205\210\346\210\221\345\223\213\347\264\204\344\270\213\344\270\255\345\234\213\351\243\262\350\214\266\351\243\237\345\227\260\345\225\262\351\263\263\347\210\252\344\275\242\345\223\213\345\217\210\346\234\203\351\251\232\343\227\216\345\222\201\346\250\243\345\230\242\345\221\242\345\260\261\345\237\272\346\234\254\344\270\212\344\275\242\345\223\213\345\224\224\346\234\203\351\251\232\345\230\205\351\243\237\347\267\212\345\232\237\343\227\216\345\222\201\346\211\200\344\273\245\344\277\202\346\257\224\350\274\203\345\244\247\345\260\261\345\277\253\345\225\262\345\233\211\346\207\211\350\251\262\345\260\261\345\224\224\346\234\203\346\213\211\351\272\265\345\222\201\346\231\256\351\200\232\345\225\246\345\224\224\344\277\202\346\234\203\345\224\224\346\234\203\344\277\202\346\227\245\346\234\254\345\230\205\345\256\232\351\243\237\345\227\260\345\225\262\345\230\242\345\230\205\350\210\226\351\240\255\347\211\271\345\210\245\345\234\250\345\222\251\345\221\242\344\275\242\351\226\213\351\213\252\351\240\255\345\227\260\345\200\213\344\272\272\344\270\213\344\275\216\345\227\260\345\200\213\344\270\273\345\273\232\345\200\213\345\270\253\345\202\205\345\221\242\344\275\242\344\270\246\345\224\224\344\277\202\346\227\245\346\234\254\344\272\272\345\223\213\345\230\205\345\222\201\345\200\213\346\225\205\344\272\213\345\245\275\345\276\227\346\204\217\344\270\213\345\205\266\345\257\246\344\275\242\345\200\213\351\274\273\350\200\201\344\272\272"

In [0]:
s

In [19]:
type(data.results[0].alternatives[0].transcript)

str

#Putting the result to Firestore

In [10]:
pip install --upgrade google-cloud-firestore

     |████████████████████████████████| 337kB 9.3MB/s 


In [0]:
from google.cloud import firestore

# Project ID is determined by the GCLOUD_PROJECT environment variable
db = firestore.Client()

In [0]:
import json

In [28]:
data

results {
  alternatives {
    transcript: "\347\276\216\345\274\217\347\204\241\351\231\220\347\204\241\351\231\220\345\257\254\351\240\273\344\272\272\344\272\272\346\234\211\344\273\275\345\245\275\346\255\241\350\277\216\345\244\247\345\256\266\346\224\266\350\201\275\344\273\212\346\227\245\345\230\205\347\204\241\351\231\220\345\257\254\351\240\273\346\210\221\345\267\262\347\266\223\347\202\272\345\244\247\345\256\266\345\217\257\344\273\245\347\225\2533d\344\273\212\346\227\245\345\221\242\346\210\221\345\260\261\346\272\226\345\202\231\345\222\227\344\270\200\351\226\223\351\244\220\345\273\263\345\220\214\344\275\240\350\254\233\344\270\213\345\233\240\347\202\272\346\210\221\350\246\272\345\276\227\344\275\240\346\234\203\351\215\276\346\204\217\351\243\237\345\230\205\345\230\242\345\232\237\345\230\205\344\275\242\346\257\217\346\254\241\351\203\275\350\251\261\346\210\221\351\215\276\346\204\217\351\243\237\345\230\205\345\230\242\345\245\275\345\256\271\346\230\223\346\2

In [31]:
import pickle 
file_result = open('010720.obj', 'w') 
pickle.dump(data, file_result)

TypeError: ignored